In [1]:
# imports
import h5py
import wandb
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
# Read the dataset
dataset_file = h5py.File("/kaggle/input/radioml2018/GOLD_XYZ_OSC.0001_1024.hdf5", "r")

# Base modulation classes
base_modulation_classes = [
    'OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
    '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
    '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
    'FM', 'GMSK', 'OQPSK'
]

# Selected modulation classes
selected_modulation_classes = [
    '4ASK', 'BPSK', 'QPSK', '16PSK', '16QAM', 'FM', 'AM-DSB-WC', '32APSK'
]

# Get the indices of selected modulation classes
selected_classes_id = [base_modulation_classes.index(cls) for cls in selected_modulation_classes]

In [3]:
# Model

class SEBlock(nn.Module):
    """ Squeeze-and-Excitation Block """
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.se(x)
        return x * scale

class MultiHeadAttention(nn.Module):
    """ Multi-Head Attention Module """
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        return attn_output

class RadioNet(nn.Module):
    def __init__(self, num_classes):
        super(RadioNet, self).__init__()

        # Separate Convolutional Pathways for I and Q
        self.q_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.i_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_size = self._get_conv_output((1, 32, 32))

        # Bidirectional LSTM with Layer Normalization
        self.lstm = nn.LSTM(self.feature_size * 2, 512, num_layers=2, 
                            batch_first=True, bidirectional=True, dropout=0.3)
        self.layer_norm = nn.LayerNorm(1024)  # Layer normalization after LSTM

        # Multi-Head Attention with multiple heads
        self.multi_head_attn = MultiHeadAttention(1024, num_heads=8)

        # Enhanced Fully Connected Layers with Dense Connections
        self.fc = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1)
        )

        self.output = nn.Linear(64, num_classes)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        output = self.q_conv(input)
        return int(torch.numel(output) / output.shape[0])

    def forward(self, i_input, q_input):
        q = self.q_conv(q_input)
        q = q.view(q.size(0), -1)

        i = self.i_conv(i_input)
        i = i.view(i.size(0), -1)

        combined = torch.cat((q, i), dim=1)
        combined = combined.unsqueeze(1)  # Add sequence dimension

        lstm_out, _ = self.lstm(combined)
        lstm_out = self.layer_norm(lstm_out)

        # Apply Multi-Head Attention
        attn_output = self.multi_head_attn(lstm_out)
        context = torch.sum(attn_output, dim=1)  # Sum up the attended output

        x = self.fc(context)
        x = self.output(x)

        return torch.log_softmax(x, dim=1)

def create_model(num_classes):
    model = RadioNet(num_classes)
    learning_rate = 0.0003
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    return model, optimizer, loss_fn, scheduler

In [4]:
# Initialize model, optimizer, and loss function
num_classes = len(selected_modulation_classes)
model, optimizer, loss_fn, scheduler = create_model(num_classes)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [5]:
# Check if multiple GPUs are available and use them if possible
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    # Wrap the model with DataParallel to parallelize across available GPUs
    model = nn.DataParallel(model)

# Set device to CUDA if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU or CPU)
model.to(device)

RadioNet(
  (q_conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): SEBlock(
      (se): Sequential(
        (0): AdaptiveAvgPool2d(output_size=1)
        (1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1))
        (2): ReLU()
        (3): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
        (4): Sigmoid()
      )
    )
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.1)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.1)

In [6]:
# Number of SNRs (from 30 SNR to 22 SNR)
N_SNR = 4 

# Initialize placeholders for data
X_data = None
y_data = None

# Loop through selected modulation classes
for id in selected_classes_id:
    # Load data slices based on indices
    X_slice = dataset_file['X'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    y_slice = dataset_file['Y'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    
    # Concatenate the slices to build the dataset
    if X_data is not None:
        X_data = np.concatenate([X_data, X_slice], axis=0)
        y_data = np.concatenate([y_data, y_slice], axis=0)
    else:
        X_data = X_slice
        y_data = y_slice

# Reshape the X_data to the required shape (e.g., 32x32 with 2 channels)
X_data = X_data.reshape(len(X_data), 32, 32, 2)

# Convert y_data to a DataFrame for easier manipulation
y_data_df = pd.DataFrame(y_data)

# Drop columns where the sum is 0 (i.e., no modulation class data in that column)
for column in y_data_df.columns:
    if sum(y_data_df[column]) == 0:
        y_data_df = y_data_df.drop(columns=[column])

# Assign the remaining columns to match the selected modulation classes
y_data_df.columns = selected_modulation_classes

# Split the dataset into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_df, test_size=0.2)

In [7]:
# Define the custom Dataset class
class RadioMLDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float().to(device)
        self.y = torch.from_numpy(y.values).float().to(device)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Set up K-fold cross-validation
n_splits = 5  # Number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Prepare the data
X = X_train
y = y_train

In [8]:
# wandb login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
# epochs
epochs = 50

In [10]:
# Initialize wandb
wandb.init(project="RadioML", name="RadioNetE50_5Fold")

# Perform K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    print(f"Fold {fold}")
    
    # Split the data
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Create Dataset objects
    train_dataset = RadioMLDataset(X_train, y_train)
    val_dataset = RadioMLDataset(X_val, y_val)
    
    # Create DataLoader objects
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model, optimizer, loss function, and scheduler
    model = RadioNet(num_classes).to(device)  # Assuming RadioNet is your model class
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
    loss_fn = torch.nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
    
    # Log model architecture
    wandb.watch(model)
    
    best_val_acc = 0
    patience = 20
    no_improve = 0
    
    for epoch in range(epochs):
        # Training loop
        model.train()
        train_loss, train_correct = 0, 0
        for batch_X, batch_y in train_loader:
            i_input = batch_X[:, :, :, 0].unsqueeze(1)
            q_input = batch_X[:, :, :, 1].unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(i_input, q_input)
            loss = loss_fn(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == batch_y.argmax(1)).sum().item()
        
        train_loss /= len(train_loader)
        train_acc = train_correct / len(train_dataset)
        
        # Validation loop
        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                i_input = batch_X[:, :, :, 0].unsqueeze(1)
                q_input = batch_X[:, :, :, 1].unsqueeze(1)
                outputs = model(i_input, q_input)
                val_loss += loss_fn(outputs, batch_y).item()
                val_correct += (outputs.argmax(1) == batch_y.argmax(1)).sum().item()
        
        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_dataset)
        
        # Log metrics to wandb
        wandb.log({
            f"fold_{fold}_epoch": epoch + 1,
            f"fold_{fold}_train_loss": train_loss,
            f"fold_{fold}_train_acc": train_acc,
            f"fold_{fold}_val_loss": val_loss,
            f"fold_{fold}_val_acc": val_acc,
            f"fold_{fold}_learning_rate": optimizer.param_groups[0]['lr']
        })
        
        # Print progress
        print(f"Fold {fold}, Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Step the learning rate scheduler
        scheduler.step(val_loss)
        
        # Early stopping and model checkpointing
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"checkpoint_fold_{fold}.pth")
            wandb.save(f"checkpoint_fold_{fold}.pth")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve == patience:
                print(f"Early stopping on fold {fold}")
                break
    
    # Log best validation accuracy for this fold
    wandb.log({f"fold_{fold}_best_val_acc": best_val_acc})

# Finish the wandb run
wandb.finish()

wandb: Currently logged in as: devcode03 (devcode03-gujarat-technological-university). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240911_153746-38auhm7w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run RadioNetE50_5Fold
wandb: ⭐️ View project at https://wandb.ai/devcode03-gujarat-technological-university/RadioML
wandb: 🚀 View run at https://wandb.ai/devcode03-gujarat-technological-university/RadioML/runs/38auhm7w


Fold 1
Fold 1, Epoch 1/50, Train Loss: 0.2110, Train Acc: 0.5732, Val Loss: 0.1971, Val Acc: 0.6042
Fold 1, Epoch 2/50, Train Loss: 0.1987, Train Acc: 0.6116, Val Loss: 0.1772, Val Acc: 0.6875
Fold 1, Epoch 3/50, Train Loss: 0.1631, Train Acc: 0.7604, Val Loss: 0.1508, Val Acc: 0.7961
Fold 1, Epoch 4/50, Train Loss: 0.1566, Train Acc: 0.7825, Val Loss: 0.1471, Val Acc: 0.8124
Fold 1, Epoch 5/50, Train Loss: 0.1541, Train Acc: 0.7909, Val Loss: 0.1467, Val Acc: 0.8122
Fold 1, Epoch 6/50, Train Loss: 0.1516, Train Acc: 0.7970, Val Loss: 0.1427, Val Acc: 0.8180
Fold 1, Epoch 7/50, Train Loss: 0.1502, Train Acc: 0.8009, Val Loss: 0.1432, Val Acc: 0.8172
Fold 1, Epoch 8/50, Train Loss: 0.1486, Train Acc: 0.8053, Val Loss: 0.1404, Val Acc: 0.8220
Fold 1, Epoch 9/50, Train Loss: 0.1456, Train Acc: 0.8080, Val Loss: 0.1380, Val Acc: 0.8218
Fold 1, Epoch 10/50, Train Loss: 0.1424, Train Acc: 0.8192, Val Loss: 0.1363, Val Acc: 0.8478
Fold 1, Epoch 11/50, Train Loss: 0.1376, Train Acc: 0.8443, Va

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:  fold_1_best_val_acc ▁
wandb:         fold_1_epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb: fold_1_learning_rate ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:     fold_1_train_acc ▁▂▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
wandb:    fold_1_train_loss █▇▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
wandb:       fold_1_val_acc ▁▃▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████████
wandb:      fold_1_val_loss █▆▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:  fold_2_best_val_acc ▁
wandb:         fold_2_epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb: fold_2_learning_rate ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:     fold_2_train_acc ▁▂▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
wandb:    fold_2_train_loss █▇▅▅▅▅▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
wandb:       fold_2_val_acc ▁▄▅▅▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇████████████
wandb:      fold_2_val_loss █▅▅▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▂▁▁▁▁▁▁▁▁▂▁▂▁▁

In [11]:
# # Initialize wandb
# wandb.init(project="RadioML", name="RadioNetE50")

# # Log model architecture
# wandb.watch(model)

# for epoch in range(epochs):
#     # Set model to training mode
#     model.train()
#     train_loss, train_correct = 0, 0
#     # Training loop
#     for batch_X, batch_y in train_loader:
#         # Split the input into I and Q components
#         i_input = batch_X[:, :, :, 0].unsqueeze(1)  # I component
#         q_input = batch_X[:, :, :, 1].unsqueeze(1)  # Q component
#         # Zero out the gradients
#         optimizer.zero_grad()
#         # Forward pass through the model
#         outputs = model(i_input, q_input)
#         # Compute loss
#         loss = loss_fn(outputs, batch_y)
#         # Backpropagation
#         loss.backward()
#         # Update model parameters
#         optimizer.step()
#         # Accumulate training loss and correct predictions
#         train_loss += loss.item()
#         train_correct += (outputs.argmax(1) == batch_y.argmax(1)).sum().item()
#     # Compute average training loss and accuracy
#     train_loss /= len(train_loader)
#     train_acc = train_correct / len(train_dataset)
#     # Validation loop (without gradient updates)
#     model.eval()
#     val_loss, val_correct = 0, 0
#     with torch.no_grad():
#         for batch_X, batch_y in test_loader:
#             # Split the input into I and Q components
#             i_input = batch_X[:, :, :, 0].unsqueeze(1)  # I component
#             q_input = batch_X[:, :, :, 1].unsqueeze(1)  # Q component
#             # Forward pass through the model
#             outputs = model(i_input, q_input)
#             # Compute validation loss
#             val_loss += loss_fn(outputs, batch_y).item()
#             # Accumulate correct predictions
#             val_correct += (outputs.argmax(1) == batch_y.argmax(1)).sum().item()
#     # Compute average validation loss and accuracy
#     val_loss /= len(test_loader)
#     val_acc = val_correct / len(test_dataset)
#     # Save loss and accuracy for later plotting
#     train_losses.append(train_loss)
#     train_accs.append(train_acc)
#     val_losses.append(val_loss)
#     val_accs.append(val_acc)

#     # Log metrics to wandb
#     wandb.log({
#         "epoch": epoch + 1,
#         "train_loss": train_loss,
#         "train_acc": train_acc,
#         "val_loss": val_loss,
#         "val_acc": val_acc,
#         "learning_rate": optimizer.param_groups[0]['lr']
#     })

#     # Print progress for this epoch
#     print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
#           f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
#     # Step the learning rate scheduler based on validation loss
#     scheduler.step(val_loss)
#     # Early stopping and model checkpointing
#     if val_acc > best_acc:
#         best_acc = val_acc
#         torch.save(model.state_dict(), path_checkpoint)  # Save the model checkpoint
#         wandb.save(path_checkpoint)  # Save the model checkpoint to wandb
#         no_improve = 0  # Reset no improvement counter
#     else:
#         no_improve += 1
#         if no_improve == patience:
#             print("Early stopping")
#             break

# # Finish the wandb run
# wandb.finish()

In [12]:
# # Plotting
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(train_accs, label='Train Accuracy')
# plt.plot(val_accs, label='Validation Accuracy')
# plt.legend()
# plt.title('Accuracy')
# plt.subplot(1, 2, 2)
# plt.plot(train_losses, label='Train Loss')
# plt.plot(val_losses, label='Validation Loss')
# plt.legend()
# plt.title('Loss')
# plt.show()

In [13]:
# # Model predictions and confusion matrix
# model.load_state_dict(torch.load(path_checkpoint))
# model.eval()
# all_preds, all_labels = [], []
# with torch.no_grad():
#     for batch_X, batch_y in test_loader:
#         i_input, q_input = batch_X[:, :, :, 0].unsqueeze(1), batch_X[:, :, :, 1].unsqueeze(1)
#         outputs = model(i_input, q_input)
#         all_preds.extend(outputs.argmax(1).cpu().numpy())
#         all_labels.extend(batch_y.argmax(1).cpu().numpy())

# cm = confusion_matrix(all_labels, all_preds)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=selected_modulation_classes)
# disp.plot()
# plt.show()

In [14]:
# Save and load model
torch.save(model.state_dict(), '/kaggle/working/CNN_LSTMmodel.pth')
loaded_model, _, _, _ = create_model(num_classes)
loaded_model.load_state_dict(torch.load('/kaggle/working/CNN_LSTMmodel.pth'))

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_23/2769534498.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of th

<All keys matched successfully>

In [15]:
# # Evaluate loaded model
# loaded_model.eval()
# loaded_model.to(device)
# correct = 0
# total = 0
# with torch.no_grad():
#     for batch_X, batch_y in test_loader:
#         i_input, q_input = batch_X[:, :, :, 0].unsqueeze(1), batch_X[:, :, :, 1].unsqueeze(1)
#         outputs = loaded_model(i_input, q_input)
#         _, predicted = torch.max(outputs.data, 1)
#         total += batch_y.size(0)
#         correct += (predicted == batch_y.argmax(1)).sum().item()

# print('Restored model, accuracy: {:.2f}%'.format(100 * correct / total))